In [1]:
import torch
from pyrlmala.agent import QNetwork
from pyrlmala.envs import BarkerEnv
from functools import partial
from scipy.stats import multivariate_normal
import numpy as np
from scipy.stats import wishart
from pyrlmala.utils import Toolbox
from pyrlmala.config import QNetworkConfigParser
import gymnasium as gym
from pyrlmala.learning.pretrain import PretrainingQNetwork

In [2]:
sample_dim = 2
critic_config_path = "config/critic.toml"

In [3]:
env = Toolbox.make_env(
    "BarkerEnv-v1.0",
    log_target_pdf=partial(
        multivariate_normal.logpdf,
        mean=np.zeros(sample_dim),
        cov=np.eye(sample_dim),
    ),
    grad_log_target_pdf=lambda x: -x,
    initial_sample=np.random.normal(size=sample_dim),
    initial_covariance=wishart.rvs(
        df=(sample_dim + 1), scale=np.eye(sample_dim)
    ).reshape(sample_dim, sample_dim),
    initial_step_size=np.random.uniform(0.1, 5.0),
    total_timesteps=1_000,
    log_mode=True,
)
envs = gym.vector.SyncVectorEnv([env])

In [4]:
qf1 = QNetwork(envs, config=QNetworkConfigParser(critic_config_path))

In [5]:
initial_sample = torch.zeros(1, sample_dim)
pretrain_q = PretrainingQNetwork(envs.envs[0].unwrapped, qf1, initial_sample, device="cpu")

In [6]:
pretrain_q.train()

Training Epochs:   0%|          | 0/1000 [00:00<?, ?it/s]